<a href="https://colab.research.google.com/github/GermanM3/GermanM3/blob/master/KOSPI_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install OpenDartReader

In [2]:
pip install finance-datareader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.0/89.0 kB 2.7 MB/s eta 0:00:00


In [34]:
import FinanceDataReader as fdr
import pandas as pd
import time
import numpy as np

In [9]:
KS = fdr.StockListing('KOSPI') # KOSPI: 940 종목
codes=KS['Code'].tolist()
codes1 = [code for code in codes if len(code) == 6 and code[-1] == '0']
filtered_codes = codes1[:30]

In [11]:
import pandas as pd
import OpenDartReader
api_key = 'your_api_key'

dart = OpenDartReader(api_key)

In [13]:
import time

# 조회 기간 설정
start_date = '20240701'  # 시작 날짜 (YYYYMMDD)
end_date = '20240813'    # 종료 날짜 (YYYYMMDD)

# 필터링할 키워드 설정
keyword = '잠정'

# 결과를 저장할 DataFrame 초기화
filtered_disclosures = pd.DataFrame()

# 종목 코드별로 반복 수행
for code in filtered_codes:
    try:
        # 공시 목록 조회
        disclosures = dart.list(code, start=start_date, end=end_date)

        # 특정 키워드가 포함된 공시만 필터링
        filtered = disclosures[disclosures['report_nm'].str.contains(keyword)]

        # 필터링된 결과를 저장
        filtered_disclosures = pd.concat([filtered_disclosures, filtered], ignore_index=True)

        # API 호출 간 지연 시간 추가 (예: 1초)
        time.sleep(1)

    except Exception as e:
        print(f"Error processing code {code}: {e}")

In [15]:
final_filtar=filtered_disclosures[filtered_disclosures['report_nm'].str.contains("연결")]

In [18]:
filtered_df=final_filtar.drop_duplicates(subset='flr_nm')
filtered_df['rcept_dt'] = pd.to_datetime(filtered_df['rcept_dt'], format='%Y%m%d')

<ipython-input-18-6a510598d137>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['rcept_dt'] = pd.to_datetime(filtered_df['rcept_dt'], format='%Y%m%d')


In [19]:
from pandas.tseries.offsets import BDay

def get_business_days_range(date, num_days):
    start_date = date - BDay(num_days)
    end_date = date + BDay(num_days)
    return start_date, end_date

# 주어진 날짜를 기준으로 범위 계산
filtered_df['start_date'], filtered_df['end_date'] = zip(*filtered_df['rcept_dt'].apply(lambda x: get_business_days_range(x, 10)))

def get_stock_data(stock_code, start_date, end_date):
    try:
        # 주가 데이터 조회
        data = fdr.DataReader(stock_code, start=start_date, end=end_date)
        return data
    except Exception as e:
        print(f"Error fetching data for stock code {stock_code}: {e}")
        return None

# 주가 데이터를 DataFrame에 추가
def fetch_data_for_all_rows(filtered_df):
    all_data = {}
    for idx, row in filtered_df.iterrows():
        stock_code = row['stock_code']
        start_date = row['start_date']
        end_date = row['end_date']

        print(f"Fetching data for stock code {stock_code} from {start_date} to {end_date}")

        stock_data = get_stock_data(stock_code, start_date, end_date)
        all_data[stock_code] = stock_data

    return all_data

# 모든 종목의 주가 데이터 조회
stock_data_dict = fetch_data_for_all_rows(filtered_df)

# 결과 출력 (예: 특정 종목의 데이터)
for stock_code, data in stock_data_dict.items():
    if data is not None:
        print(f"Data for stock code {stock_code}:")
        print(data.head())

<ipython-input-19-a0d40f106abc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['start_date'], filtered_df['end_date'] = zip(*filtered_df['rcept_dt'].apply(lambda x: get_business_days_range(x, 10)))
<ipython-input-19-a0d40f106abc>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['start_date'], filtered_df['end_date'] = zip(*filtered_df['rcept_dt'].apply(lambda x: get_business_days_range(x, 10)))


Fetching data for stock code 005930 from 2024-07-17 00:00:00 to 2024-08-14 00:00:00
Fetching data for stock code 000660 from 2024-07-11 00:00:00 to 2024-08-08 00:00:00
Fetching data for stock code 373220 from 2024-07-11 00:00:00 to 2024-08-08 00:00:00
Fetching data for stock code 207940 from 2024-07-10 00:00:00 to 2024-08-07 00:00:00
Fetching data for stock code 005380 from 2024-07-11 00:00:00 to 2024-08-08 00:00:00
Fetching data for stock code 068270 from 2024-07-24 00:00:00 to 2024-08-21 00:00:00
Fetching data for stock code 000270 from 2024-07-12 00:00:00 to 2024-08-09 00:00:00
Fetching data for stock code 105560 from 2024-07-09 00:00:00 to 2024-08-06 00:00:00
Fetching data for stock code 055550 from 2024-07-12 00:00:00 to 2024-08-09 00:00:00
Fetching data for stock code 005490 from 2024-07-11 00:00:00 to 2024-08-08 00:00:00
Fetching data for stock code 035420 from 2024-07-26 00:00:00 to 2024-08-23 00:00:00
Fetching data for stock code 028260 from 2024-07-17 00:00:00 to 2024-08-14 0

In [20]:
data_frames = []
for stock_code, temp in stock_data_dict.items():
    # 'Close' 열만 추출하고 'Date'를 인덱스로 설정
    temp = temp[['Close']].rename(columns={'Close': stock_code})
    data_frames.append(temp)

# 모든 데이터프레임을 병합
result_temp = pd.concat(data_frames, axis=1)

In [21]:
import pandas as pd

# 수익률 계산 함수
def calculate_returns(result_temp, filtered_df, days):
    """
    특정 날짜 전후로 수익률을 계산합니다.
    result_temp: 종목별 종가 데이터프레임 (Date 인덱스, 종목코드 컬럼)
    filtered_df: rcept_dt와 종목코드를 포함하는 데이터프레임
    days: 기준 날짜 전후의 일 수
    """
    returns_dict = {}

    # 각 종목에 대해
    for idx, row in filtered_df.iterrows():
        stock = row['stock_code']
        rcept_date = row['rcept_dt']

        # 기준 날짜 전후로 일정 기간 데이터 선택
        temp = result_temp.loc[rcept_date - pd.Timedelta(days=days): rcept_date + pd.Timedelta(days=days), stock]

        # 수익률 계산
        temp_returns = temp.pct_change() * 100  # 백분율로 변환

        # 결과를 저장
        returns_dict[stock] = temp_returns

    # 결과를 데이터프레임으로 변환
    returns_df = pd.DataFrame(returns_dict)

    return returns_df

# 예시: 기준 날짜 전후 10일 동안의 수익률 계산
days = 10
returns_df = calculate_returns(result_temp, filtered_df, days)

# 결과 확인
print(returns_df)

               005930     000660    373220    207940    005380    068270  \
Date                                                                       
2024-07-15        NaN        NaN       NaN       NaN       NaN       NaN   
2024-07-16        NaN   1.304348 -3.347280  0.610501  2.234637       NaN   
2024-07-17        NaN  -5.364807 -2.886003  3.276699 -2.185792       NaN   
2024-07-18        NaN  -3.628118 -0.445765 -0.117509 -2.979516       NaN   
2024-07-19        NaN  -1.411765  3.134328 -1.764706 -0.767754       NaN   
2024-07-22        NaN  -2.147971 -4.920405  1.556886 -1.353965       NaN   
2024-07-23   1.084337   0.000000 -0.761035  0.000000  3.333333       NaN   
2024-07-24  -2.264601   1.707317 -1.380368  4.127358 -1.897533       NaN   
2024-07-25  -1.951220  -8.872902  3.421462 -1.359003 -2.707930       NaN   
2024-07-26   0.621891   0.947368 -2.255639  5.051665 -3.180915       NaN   
2024-07-29   0.370828   1.981230  1.384615  2.732240  4.928131       NaN   
2024-07-30  

In [30]:
returns_df_T=returns_df.T

In [31]:
returns_df_T.index

Index(['005930', '000660', '373220', '207940', '005380', '068270', '000270',
       '105560', '055550', '005490', '035420', '028260', '006400', '051910',
       '012330', '329180', '086790', '035720', '003670', '259960', '066570',
       '012450', '009540', '033780', '003550', '015760'],
      dtype='object')

In [32]:
percent_df=returns_df_T/100

In [35]:
def calculate_cumulative_returns(row):
    # 유효한 수익률(Non-NaN)만 선택
    valid_returns = row.dropna()

    # 중간 지점 계산
    mid_point = len(valid_returns) // 2

    # 중간 이전의 누적 수익률
    pre_event_cumulative = np.prod(1 + valid_returns[:mid_point]) - 1

    # 중간 이후의 누적 수익률
    post_event_cumulative = np.prod(1 + valid_returns[mid_point:]) - 1

    return pre_event_cumulative, post_event_cumulative

# 각 행에 대해 계산 적용
percent_df[['Pre-Event Return', 'Post-Event Return']] = percent_df.apply(calculate_cumulative_returns, axis=1, result_type="expand")
percent_df[['Pre-Event Return', 'Post-Event Return']] .round(4)

Date,Pre-Event Return,Post-Event Return
005930,0.0108,-0.1097
000660,-0.0935,-0.1693
373220,-0.1032,0.0451
207940,0.0781,0.0317
005380,-0.0372,-0.0561
068270,-0.1268,0.0553
000270,-0.0643,-0.1626
105560,-0.0128,-0.0165
055550,0.0361,-0.0312
005490,-0.1023,0.0289


In [37]:
percent_df

Date,2024-07-15 00:00:00,2024-07-16 00:00:00,2024-07-17 00:00:00,2024-07-18 00:00:00,2024-07-19 00:00:00,2024-07-22 00:00:00,2024-07-23 00:00:00,2024-07-24 00:00:00,2024-07-25 00:00:00,2024-07-26 00:00:00,...,2024-08-02 00:00:00,2024-08-05 00:00:00,2024-08-06 00:00:00,2024-08-07 00:00:00,2024-08-08 00:00:00,2024-08-09 00:00:00,2024-08-12 00:00:00,2024-08-13 00:00:00,Pre-Event Return,Post-Event Return
005930,NaN,NaN,NaN,NaN,NaN,NaN,0.010843,-0.022646,-0.019512,0.006219,...,-0.042118,-0.103015,0.015406,0.030345,-0.017403,0.017711,NaN,NaN,0.010843,-0.109654
000660,NaN,0.013043,-0.053648,-0.036281,-0.014118,-0.021480,0.000000,0.017073,-0.088729,0.009474,...,-0.103983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.093478,-0.169305
373220,NaN,-0.033473,-0.028860,-0.004458,0.031343,-0.049204,-0.007610,-0.013804,0.034215,-0.022556,...,0.007496,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.103208,0.045101
207940,NaN,0.006105,0.032767,-0.001175,-0.017647,0.015569,0.000000,0.041274,-0.013590,0.050517,...,-0.015135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.078144,0.031710
005380,NaN,0.022346,-0.021858,-0.029795,-0.007678,-0.013540,0.033333,-0.018975,-0.027079,-0.031809,...,-0.037475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.037244,-0.056093
068270,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.032000,-0.057335,-0.002740,0.082418,-0.006599,0.001533,0.020408,-0.037000,-0.126794,0.055342
000270,NaN,NaN,-0.015460,-0.009091,-0.030025,-0.003439,0.039689,-0.017427,-0.028716,-0.025217,...,-0.044603,-0.100840,NaN,NaN,NaN,NaN,NaN,NaN,-0.064280,-0.162609
105560,NaN,-0.018670,0.000000,0.003567,0.004739,0.020047,0.020809,-0.041903,-0.007092,0.046429,...,-0.057758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.012835,-0.016548
055550,NaN,NaN,0.022814,0.009294,-0.016575,0.029963,0.001818,-0.005445,-0.005474,0.064220,...,-0.059308,-0.075306,NaN,NaN,NaN,NaN,NaN,NaN,0.036122,-0.031193
005490,NaN,-0.033679,-0.020107,0.002736,0.010914,-0.022942,-0.019337,-0.023944,0.025974,0.002813,...,-0.016552,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.102332,0.028860


In [39]:
KS1=KS.copy()
KS1.set_index('Code', inplace=True)
percent_df['Name'] = KS1.loc[percent_df.index, 'Name']


In [41]:
percent_df[['Name', 'Pre-Event Return', 'Post-Event Return']] .round(4)

Date,Name,Pre-Event Return,Post-Event Return
005930,삼성전자,0.0108,-0.1097
000660,SK하이닉스,-0.0935,-0.1693
373220,LG에너지솔루션,-0.1032,0.0451
207940,삼성바이오로직스,0.0781,0.0317
005380,현대차,-0.0372,-0.0561
068270,셀트리온,-0.1268,0.0553
000270,기아,-0.0643,-0.1626
105560,KB금융,-0.0128,-0.0165
055550,신한지주,0.0361,-0.0312
005490,POSCO홀딩스,-0.1023,0.0289
